In [1]:
import torch
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet34, mobilenet_v2
from torchvision import transforms
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier, VotingClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import joblib
import random

# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/attack/pickle/cifar10/mobilenetv2/shadow.p'

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
print(type(dataset), len(dataset))

# split the dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

cuda:0
<class 'list'> 30000


# shadow model

In [ ]:
# model parameters
n_classes = 10
n_epochs = 50
batch_size = 64
learning_rate = 0.001
weight_decay = 1e-4  # parameter for L2 regularization


# load target model for shadow model
shadow_model = mobilenet_v2(num_classes=n_classes).to(device)
state_dict = torch.load('/content/drive/My Drive/Colab Notebooks/attack/models/mobilenetv2_cifar10.pth', map_location=device)
shadow_model.load_state_dict(state_dict['net'])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(shadow_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# model training
shadow_model.train()
for epoch in range(n_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = shadow_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    scheduler.step()

    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {running_loss/len(train_loader)}")
    # save the model after each 5 epochs
    if (epoch+1) % 5 == 0:
        torch.save(shadow_model.state_dict(), f"/content/drive/My Drive/Colab Notebooks/attack/my_models/cifar10_mobilenetv2/shadow_model_epoch_{epoch+1}.pt")

Epoch 1/50, Loss: 0.9251036982536316
Epoch 2/50, Loss: 0.7274957598050436
Epoch 3/50, Loss: 0.6376436343193054
Epoch 4/50, Loss: 0.5812655079364777
Epoch 5/50, Loss: 0.5291875459353129
Epoch 6/50, Loss: 0.4873831107616425
Epoch 7/50, Loss: 0.4482312090396881
Epoch 8/50, Loss: 0.41216957521438596
Epoch 9/50, Loss: 0.38252473751703897
Epoch 10/50, Loss: 0.37294114752610524
Epoch 11/50, Loss: 0.2279144227306048
Epoch 12/50, Loss: 0.17302101356784502
Epoch 13/50, Loss: 0.13881408727169037
Epoch 14/50, Loss: 0.11987580218414466
Epoch 15/50, Loss: 0.10222961216668287
Epoch 16/50, Loss: 0.08177369946489732
Epoch 17/50, Loss: 0.0742864736393094
Epoch 18/50, Loss: 0.06709832676996787
Epoch 19/50, Loss: 0.0588930714726448
Epoch 20/50, Loss: 0.05296866709676882
Epoch 21/50, Loss: 0.043433804099758465
Epoch 22/50, Loss: 0.04374569136276841
Epoch 23/50, Loss: 0.03924062302398185
Epoch 24/50, Loss: 0.036975142516816654
Epoch 25/50, Loss: 0.03836765003514787
Epoch 26/50, Loss: 0.033815564900636676
Ep

In [24]:
# load trained shadow model
n_classes = 10
criterion = nn.CrossEntropyLoss()

shadow_model = mobilenet_v2(num_classes=n_classes).to(device)
shadow_model.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/attack/my_models/cifar10_mobilenetv2/shadow_model_epoch_50.pt"))

<All keys matched successfully>

In [25]:
# model evaluation
test_loss = 0.0
correct = 0
total = 0
shadow_model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = shadow_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1) # get the class index
        total += labels.size(0) # total samples
        correct += (predicted == labels).sum().item() # correctly predicted samples

print(f'Test Loss: {test_loss/len(test_loader)}, Accuracy: {100 * correct / total}%')

Test Loss: 1.2281129594813003, Accuracy: 74.3%


#### Attack Model Training

In [26]:
# method for generating attack dataset for given model and dataset
def generate_attack_data(model, data_loader, label):
    attack_data = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = nn.functional.softmax(outputs, dim=1)
            for prob in probabilities:
                attack_data.append((prob.cpu().numpy(), label))
    return attack_data

# generate attack data
member_data = generate_attack_data(shadow_model, train_loader, 1) # member label as 1
non_member_data = generate_attack_data(shadow_model, test_loader, 0) # non-member label as 0
attack_data = member_data + non_member_data

# prepare the data for training attack model
np.random.shuffle(attack_data)
X = np.array([x[0] for x in attack_data])
y = np.array([x[1] for x in attack_data])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [29]:
# train attack model
attack_model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=12,
    learning_rate=0.03,
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    subsample=0.9,
    colsample_bytree=0.5,
    reg_alpha=0.1,
    reg_lambda=4.0,
    early_stopping_rounds=30,
    scale_pos_weight=0.2,
    tree_method = 'hist',
    device = 'gpu'
)

attack_model.fit(X_train, y_train,
                 eval_set=[(X_train, y_train), (X_test, y_test)],
                 verbose=False)

y_pred = attack_model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f"Attack Model Accuracy on train set: {accuracy}")

# evaluate the attack model on the test data
y_pred = attack_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Attack Model Accuracy on test set: {accuracy}")

Attack Model Accuracy on train set: 0.9530833333333333
Attack Model Accuracy on test set: 0.7686666666666667


#### Target Model

In [30]:
# load the target model
MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/attack/models/mobilenetv2_cifar10.pth'

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_model = mobilenet_v2(num_classes=10).to(device)

state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

print(target_model)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [31]:
# loading evaluation data
with open('/content/drive/My Drive/Colab Notebooks/attack/pickle/cifar10/mobilenetv2/eval.p', 'rb') as f:
    eval_dataset = pickle.load(f)

# prepare the data for the evaluation
eval_images = [item[0] for item in eval_dataset]
eval_labels = [item[1] for item in eval_dataset]
eval_membership_status = [item[2] for item in eval_dataset]

eval_images_tensor = torch.stack(eval_images)
eval_labels_tensor = torch.tensor(eval_labels)
eval_dataset_tensor = TensorDataset(eval_images_tensor, eval_labels_tensor)
eval_loader = DataLoader(eval_dataset_tensor, batch_size=64, shuffle=False, num_workers=2)


In [32]:
#  Generate evaluation data for attack model using target model
probabilities = []
target_model.eval()
with torch.no_grad():
    for inputs, _ in eval_loader:
        inputs = inputs.to(device)
        outputs = target_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        probabilities.extend(probs.cpu().numpy())

X_eval = np.array(probabilities)
y_eval_true = np.array(eval_membership_status)

In [39]:
# evaluate the attack model
y_eval_pred = attack_model.predict(X_eval)
accuracy = accuracy_score(y_eval_true, y_eval_pred)
print(f"Membership Inference Accuracy on evaluation data: {accuracy}")


Membership Inference Accuracy on evaluation data: 0.69


In [40]:
joblib.dump(attack_model, '/content/drive/My Drive/Colab Notebooks/attack/attack_models/mobilenetv2_cifar10/attack_model_69.pkl')

['/content/drive/My Drive/Colab Notebooks/attack/attack_models/mobilenetv2_cifar10/attack_model_69.pkl']